# 4.4 Logic Programming

## 4.55

```scheme
;; 1
(supervisor ?x (Bitdiddle Ben))

;; 2
(job ?x (accounting . ?type))

;; 3
(address ?x (Sumerville . ?y))
```

## 4.56

```scheme
;; 1
(and (supervisor ?person (Bitdiddle Ben))
     (address ?person . ?street))
;; 2
(and (salary (Bitdiddle Ben) ?x)
     (salary ?person ?amount)
     (lisp-value < ?amount ?x))

;; 3
(and (supervisor ?person ?supervisor)
     (job ?supervisor ?job)
     (not (job ?supervisor (computer . ?type))))
```

## 4.57

```scheme
(rule (can-replace ?person1 ?person2)
      (and
       (or
        ;; person1 does same job as person2
        (and (job ?person1 ?job) 
             (job ?person2 ?job))
        ;; OR someone who does person1's job can also do person2's job
        (and (job ?person1 ?job1)
             (job ?person2 ?job2)
             (can-do-job ?j1 ?j2)))
       ;; person1 and person2 are not the same person
       (not (same ?person1 ?person2))))

;; 1
(can-replace ?person (Fect Cy D))

;; 2 

(and (can-replace ?person1 ?person2)
     (salary ?person1 ?salary1)
     (salary? ?person2 ?salary2)
     (lisp-value > salary2 salary1))
```

## 4.58

```scheme
(rule (big-shot ?person ?division)
      (and 
       ;; person works in given division
       (job ?person (?division . ?type))
       (supervisor ?person ?supervisor)
       (job ?supervisor (?sup-division . ?sup-type))
       ;; persons superervisor does not work in given divistion
       (not (same ?division ?sup-division))))
```

## 4.59

```scheme
;; 1
(meeting ?division (Friday ?time))

;; 2
(rule (meeting-time ?person ?day-and-time)
      (or (meeting whole-company ?day-and-time)
          (and
           (job ?person (?division . ?type))
           (meeting ?division ?day-and-time))))

;; 3
(meeting-time (Hacker Alyssa P) (Wednesday ?time))
```

## 4.60

The `lives-near` rule is commutative, so the following two queries will both return the same *values* but in the opposite order:

```scheme
(lives-near ?person-1 ?person-2)
(lives-near ?person-2 ?person-1)
```

This is because `lives-near` is implemented using an `and` expression, which is in itself commutative:

```scheme
(rule (lives-near ?person-1 ?person-2)
      ;; arguments to and could be in any order
      (and (address ?person-1 
                    (?town . ?rest-1))
           (address ?person-2 
                    (?town . ?rest-2))
           (not (same ?person-1 ?person-2))))
```

This poses no restrictions on the values that `?person-1` and `?person-2` can take, so every *combination* that the query succeeds for is returned.

To produce a list of *unique pairs* of people who live near each other, a futher restriction can be added to the `lives-near` rule that enforces an **ordering** of the pairs. For example, enforcing alphabetical ordering of `person-1` and `person-2` in the pair will mean only one combination of the two people is returned. For example, `(Hacker Alyssa P) (Fect Cy D))` is alphabetically *less* than `((Fect Cy D) (Hacker Alyssa P))`.

In [21]:
;; from previous chapter
(define (accumulate op initial sequence)
  (if (null? sequence)
      initial
      (op (car sequence)
          (accumulate op
                      initial
                      (cdr sequence)))))

;; convert a list of symbols into a single string
(define (symbol-list->string lst)
  (accumulate string-append "" (map symbol->string lst)))

;; compare two lists of symbols to determine if
;; lst1 is alphabetically less than lst2
(define (symbol-list<? lst1 lst2)
  (string<? (symbol-list->string lst1)
            (symbol-list->string lst2)))

(symbol-list<? '(Hacker Alyssa P) '(Fect Cy D))

#f

In [22]:
(symbol-list<?  '(Fect Cy D) '(Hacker Alyssa P))

#t

```scheme
(rule (lives-near ?person-1 ?person-2)
      (and (address ?person-1 
                    (?town . ?rest-1))
           (address ?person-2 
                    (?town . ?rest-2))
           (not (same ?person-1 ?person-2))
           (lisp-value symbol-list>? ?person-1 ?person-1)))
```

## 4.61

```scheme
(rule (?x next-to ?y in (?x ?y . ?u)))
(rule (?x next-to ?y in (?v . ?z))
      (?x next-to ?y in ?z))

(?x next-to ?y in (1 (2 3) 4))
;; ((2 3) next-to 4 in (1 (2 3) 4))
;; (1 next-to (2 3) in (1 (2 3) 4))

(?x next-to 1 in (2 1 3 1))
;; (2 next-to 1 in (2 1 3 1))
;; (3 next-to 1 in (2 1 3 1))
```

## 4.62 - TODO Test after implemented query evaluator

#### Expected results from examples:

```scheme
(last-pair (3) ?x)
;; (last-pair (3) (3))

(last-pair (1 2 3) ?x)
;; (last-pair (1 2 3) (3))

(last-pair (2 ?x) (3))
;; (last-pair (2 3) (3))

;; not possible -> infinite possible solutions:
(last-pair ?x (3))
;; (last-pair (3) (3))
;; (last-pair (1 3) (3))
;; (last-pair (2 3) (3))
;; (last-pair (3 3) (3))
;; ...
```

#### `last-pair` rules:

- For a single element list `x`, `last-pair` is `x`
- For any `x`, `y`, `z`, `last-pair` of `(cons x y)` is `z` *if* `last-pair` of `y` is `z`

```scheme
(rule (last-pair (?x) ?x))
(rule (last-pair (?x . ?y) ?z)
      (last-pair ?y (?z)))
```

## 4.63 TODO Test after implemented query evaluator

```scheme
(rule (grandson ?g ?s)
      (and (son ?f ?s)
           (son ?g ?f)))
(rule (son ?w ?s)
      (and (wife ?m ?w)
           (son ?w ?s)))
```

## 4.64

Louis' `outranked-by` rule

```scheme
(rule (outranked-by ?staff-person ?boss)
      (or (supervisor ?staff-person ?boss)
          (and (outranked-by ?middle-manager
                             ?boss)
               (supervisor ?staff-person
                           ?middle-manager))))
```

Method for applying a rule described in 4.4.2:
1. Unify the query with the conclusion of the rule to form, if successful, an extension of the original frame.
2. Relative to the *extended* frame, evaluate the query formed by the body of the rule.

Unify the query with the conclusion of the rule:
```scheme
;; query
(outranked-by (Bitdiddle Ben) ?who)

;; conclusion of rule
(outranked-by ?staff-person ?boss)
```

This `succeeds`, extending the original frame with a binding from `?staff-person` -> `(Bitdiddle Ben)` and from `?boss` -> `?who`:

```scheme
;; body of rule
(or (supervisor ?staff-person ?boss)
    (and (outranked-by ?middle-manager
                       ?boss)
         (supervisor ?staff-person
                     ?middle-manager)))

;; with the extended binding from step 1 becomes
(or (supervisor (Bitdiddle Ben) ?who)
    (and (outranked-by ?middle-manager
                       ?who)
         (supervisor (Bitdiddle Ben)
                     ?middle-manager)))
```

Evaluating the `outranked-by` query within the `and` expression:

```scheme
;; query
(outranked-by ?middle-manager ?who)

;; conclusion of rule
(outranked-by ?staff-person ?boss)
```

Again, this succeeds, extending the original frame with a binding from `?staff-person` -> `?middle-manager` and from `?boss` -> `?who`.

```scheme
;; body of rule within extended binding
(or (supervisor ?middle-manager ?who)
    (and (outranked-by ?middle-manager
                       ?who)
         (supervisor ?middle-manager
                     ?middle-manager)))
```

The `outranked-by` query within the `and` expression is evaluated again, binding `?staff-person` -> `?middle-manager` and `?boss` -> `?who`, repeating this infinitely.

The original definition of `outranked-by` from 4.4.1 doesn't cause an infinite loop because the sub-queries within the `and` expression are in opposite order. The `supervisor` query is evaluated first which creates bindings for `?staff-person` and `?middle-manager` which restrict the search space for the recursive `outranked-by` query.

## 4.65

`wheel` rule from 4.4.1:
```scheme
(rule (wheel ?person)
      (and (supervisor ?middle-manager
                       ?person)
           (supervisor ?x ?middle-manager)))
```

The query `(wheel ?who)` results in Oliver Warbucks being listed four times because the database contains entries corresponding to four middle managers who have Oliver Warbucks as a manager. The `wheel` query does not put any restriction on uniqueness of results and will just return all matches found in the database. Using the database defined in 4.4.1, Oliver Warbucks is the manager of the following middle managers:
- Ben Bitdiddle 
- Eben Scrooge

Ben Bitdiddle, however, has entries which describe 3 different people for whom he is the middle manager:
- Lem E Tweakit
- Cy D Fect
- Alyssa P Hacker

There is one entry for whom Ben Bitdiddle is the `wheel`:
- Alyssa P Hacker
    - Middle manager of Louis Reasoner

Each of these will return Oliver Warbucks as being the `wheel`, as he is the manager of Ben Bitdiddle.